In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. Загрузка и предварительная обработка данных
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.CIFAR10(root='./cifar', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./cifar', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [21]:


# 2. Определение модели
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 3. Обучение модели
for epoch in range(5):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# 4. Оценка производительности
correct = 0
total = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")


Epoch 1, Loss: 1.3561947345733643
Epoch 2, Loss: 1.7295626401901245
Epoch 3, Loss: 1.09938383102417
Epoch 4, Loss: 1.1105139255523682
Epoch 5, Loss: 0.8446274995803833
Accuracy of the network on the 10000 test images: 50.02%


In [18]:
torch.max(outputs.data, 1)

torch.return_types.max(
values=tensor([ 3.0403, 18.4345,  2.6203,  6.5708,  3.3702,  2.2428,  2.2728,  7.4451,
         2.2863,  4.1873, 10.1391,  2.9424,  4.4234,  3.1299,  7.6714,  4.2203]),
indices=tensor([6, 1, 9, 8, 1, 3, 3, 7, 4, 4, 5, 7, 7, 4, 7, 8]))

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses

# 1. Загрузка и предварительная обработка данных
cifar = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

2023-08-31 12:17:06.663435: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 12:17:06.702634: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 12:17:06.703441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 12:17:07.529659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
test_images.shape

(10000, 32, 32, 3)

In [19]:
test_images.max()

255

In [15]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
%tensorboard --logdir /home/gea/TMS_projects/logs/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-08-31 11:13:36.854251: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 11:13:36.892221: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 11:13:36.892547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 11:13:37.516968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gea/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/gea/.local/lib/python3.8/site-packages/tensorboard_data

In [12]:
%reload_ext tensorboard

In [16]:
import datetime

In [3]:
rm -rf ./logs/

In [6]:


train_images = train_images.reshape((50000, 32, 32, 3))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

# 2. Определение модели
model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 3)))
model.add(layers.Dense(500, activation='sigmoid'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# 3. Обучение модели
model.fit(train_images, train_labels, epochs=10, batch_size=32)

# 4. Оценка производительности
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Accuracy on the test set: {test_acc * 100:.2f}%")


Epoch 1/10


2023-08-31 12:17:12.508880: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 14s 9ms/step - loss: 1.8797 - accuracy: 0.3291
Epoch 2/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.7149 - accuracy: 0.3926
Epoch 3/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.6275 - accuracy: 0.4240
Epoch 4/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.5651 - accuracy: 0.4461
Epoch 5/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.5231 - accuracy: 0.4614
Epoch 6/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4862 - accuracy: 0.4731
Epoch 7/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4488 - accuracy: 0.4876
Epoch 8/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4197 - accuracy: 0.4962
Epoch 9/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.3926 - accuracy: 0.5072
Epoch 10/10
313/313 [==============================] - 1s 4ms/step - loss: 1.4147 - 

In [7]:
train_images = train_images.reshape((50000, 32, 32, 3))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

# 2. Определение модели



model = models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=train_images.shape[1:], activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax'),
])


model.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# 3. Обучение модели
model.fit(train_images, train_labels, epochs=5, batch_size=32)

# 4. Оценка производительности
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Accuracy on the test set: {test_acc * 100:.2f}%")

Epoch 1/5


2023-08-31 12:19:26.321092: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 80s 51ms/step - loss: 2.3037 - accuracy: 0.1025
Epoch 2/5
1563/1563 [==============================] - 80s 51ms/step - loss: 2.3036 - accuracy: 0.0978
Epoch 3/5
1563/1563 [==============================] - 79s 51ms/step - loss: 2.3034 - accuracy: 0.0996
Epoch 4/5
1563/1563 [==============================] - 81s 52ms/step - loss: 2.3033 - accuracy: 0.0995
Epoch 5/5
313/313 [==============================] - 4s 11ms/step - loss: 2.3029 - accuracy: 0.1000
Accuracy on the test set: 10.00%


In [17]:
%tensorboard --logdir ./logs/fit/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-08-31 12:35:27.621364: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 12:35:27.659735: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 12:35:27.660031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 12:35:28.256204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gea/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/gea/.local/lib/python3.8/site-packages/tensorboard_data